In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
def create_target(t):
        target_vector = np.zeros(10)
        for i in range(10):
            if i == t:
                target_vector[i] = 1
        return target_vector
    
def hsig(x):
    z = np.clip(x, 0, 1)
    return np.copy(z)

def d_hsig(x):
    z = (x > 0) & (x < 1)
    return np.copy(z)

def relu(x):
    y = x * (x > 0)
    return np.copy(y)

def softmax(y):
    e_y = np.exp(y - np.max(y))
    return e_y / e_y.sum()

In [3]:
digits = datasets.load_digits()
data = digits.data
targets = digits.target

inputs = data - np.mean(data)
inputs = inputs/(np.std(data))

In [21]:
n_x = 64
n_h = 50
n_y = 10

epsilon = 0.1

W1 = np.random.normal(0, (4/(n_x + n_y)), (n_x, n_h))
W2 = np.random.normal(0, (4/(n_h)), (n_h, n_y))

bh = np.random.normal(0, 4/(n_x + n_y), n_h)
by = np.random.normal(0, 4/(n_h), n_y)

alpha1 = 0.01
alpha2 = 0.005
beta = 1

h = np.random.uniform(0, 1, n_h)
y = np.random.uniform(0, 1, n_y)

for ex in range(10000):
    rnd = np.random.randint(0, 1497)
    x = inputs[rnd] + 0.1 * np.random.rand(64)
    t = create_target(targets[rnd])
    h = np.random.uniform(0, 1, n_h)
    y = np.random.uniform(0, 1, n_y)
    
    #Free Phase
    for itr in range(100):
        dh = d_hsig(h) * (np.dot(x, W1) + np.dot(y, W2.T) + bh) - h
        dy = d_hsig(y) * (np.dot(h, W2) + by) - y
        
        h = hsig(h + epsilon * dh)
        y = hsig(y + epsilon * dy)
        
    h_free = np.copy(h)
    y_free = np.copy(y)
    
    #Weakly Clamped Phase
    for itr in range(20):
        dy = d_hsig(y) * (np.dot(h, W2) + by) - y + beta * (t - y)
        dh = d_hsig(h) * (np.dot(x, W1) + np.dot(y, W2.T) + bh) - h
        
        h = hsig(h + epsilon * dh)
        y = hsig(y + epsilon * dy)
        
    h_clamped = np.copy(h)
    y_clamped = np.copy(y)
    
    W1 += alpha1 * (1/beta) * (np.outer(x, h_clamped) - np.outer(x, h_free))
    W2 += alpha2 * (1/beta) * (np.outer(h_clamped, y_clamped) - np.outer(h_free, y_free))
    
    if ex % 100 == 0:
        print(np.dot(t - y_free, t - y_free))
        
    if ex % 5000 == 4999:
        alpha1 /= 10
        alpha2 /= 10

1.0657443184038147
0.8156431225150086
0.9538805702301232
0.5712746606859086
0.6006255214197636
0.3140642814754854
0.5543244751795378
0.4073431488332192
0.3275842177408226
0.18134664321699884
0.35484950448431163
0.17503657745084977
0.24621165048477944
0.05555049837276933
0.26245072976921335
0.00721929034468242
0.22794007721706322
1.0073491434024986
0.010026698879671555
0.04149436414450918
1.4313809711902428
0.08350748795869564
0.07814798275463447
0.2551819325461274
1.0021029425675416
0.010451393689239074
0.17376070850555753
0.24874105459310297
0.008715893938424882
0.3253700731892186
0.022162263297846214
0.0033580488411244958
0.11329343350205487
0.0005577434086581132
0.02957820449277595
0.23085676477208464
0.02242280279668752
0.006504398509915554
0.0
2.8199120270343678e-05
0.09161439288237339
0.04511081455378611
5.874219933628081e-05
0.011627645877333621
0.00021707122808314934
0.17259774705681807
0.0002838635825742092
0.08955936836071117
0.3477373284342486
0.08520688222704972
0.035280354

In [22]:
score = 0
for test in range(200):
    rnd = np.random.randint(1497, 1797)
    x = inputs[rnd]
    t = create_target(targets[rnd])
    h = np.random.uniform(0, 1, n_h)
    y = np.random.uniform(0, 1, n_y)
    
    #Free Phase
    for itr in range(100):
        dh = d_hsig(h) * (np.dot(x, W1) + np.dot(y, W2.T) + bh) - h
        dy = d_hsig(y) * (np.dot(h, W2) + by) - y
        
        h = hsig(h + epsilon * dh)
        y = hsig(y + epsilon * dy)
        
    h_free = np.copy(h)
    y_free = np.copy(y)
    
    if np.argmax(y_free) == targets[rnd]:
        score += 1
    
print(score/200)

0.895
